In [190]:
import pandas as pd
import pickle
import numpy as np
import dissertation.tools.utils as utils
import librosa
import matplotlib.pyplot as plt
import subprocess
import glob
import scipy.signal as signal
import os

from keras.models import model_from_json
from skimage.transform import resize
from speech2ult.tools.tal_io import read_ultrasound_tuple
from speech2ult.tools.transform_ultrasound import transform_ultrasound
from speech2ult.tools.voice_activity_detection import detect_voice_activity, separate_silence_and_speech


In [191]:
def resize_ult(ult, n_scan, n_echo):
    """
    resize the ult to new size n_scan x n_echo. adapted from ultrasuite tools core.py
    :return: ult
    """

    resized = []
    for i, image in enumerate(ult):
        temp = resize(image, output_shape=(n_scan, n_echo), order=0, mode='reflect', clip=False,
                      preserve_range=True, anti_aliasing=False)
        temp = temp.round().astype(int)
        resized.append(temp)

    ult = np.array(resized)
    return ult

def load_ult(file):
    """load, downsample and resize ultrasound, returned with the sync offset in seconds"""
    ult, param_file = read_ultrasound_tuple(file, shape='3d', cast=None, truncate=None)
    offset = param_file['TimeInSecsOfFirstFrame']
    # current n_frames * target_fps/current_fps
    target_frames = int(ult.shape[0] * 60 / param_file['FramesPerSec'])
    ult = utils.resize(ult, target_frames)
    # resize dimensions
    ult = resize_ult(ult, 64, 128)


    return ult, offset

def clean_data(data):
    """Clean the dlc pandas dataset and return np array of features"""
    body_parts = data.loc['bodyparts'].values + '_' + data.loc['coords'].values
    data.columns = body_parts
    data.drop(index=['bodyparts', 'coords'], inplace=True)
    del_col = [col for col in data.columns if col.endswith('likelihood')]
    data.drop(columns=del_col, inplace=True)
    # get rid of columns for hyoid, mandible and short tendon as well
    if 'hyoid_x' in data.columns:
        del_parts = ['hyoid_x', 'hyoid_y','mandible_x', 'mandible_y', 'shortTendon_x', 'shortTendon_y']
        data.drop(columns=del_parts, inplace=True)

    return data.values

def apply_butter(dlc_feats):
    """apply butterworth filter to each column (body part)"""
    filtered = np.empty_like(dlc_feats)
    fc = 10  # Cut-off frequency of the filter
    w = fc / (60 / 2)  # Normalize the frequency
    b, a = signal.butter(5, w, 'low')

    for col in range(dlc_feats.shape[1]):
        filtered[:, col] = signal.filtfilt(b, a, dlc_feats[:, col])

    return filtered

def apply_medfilt(dlc_feats):
    """apply median filter to each column (body part)"""
    filtered = np.empty_like(dlc_feats)
    for col in range(dlc_feats.shape[1]):
        filtered[:, col] = signal.medfilt(dlc_feats[:, col], 7)

    return filtered

def load_dlc_ult(basename, path, butter=False, med=False):
    """
    read in dlc ult. takes optional filter args
    """
    csv_file = glob.glob(os.path.join(path, f'day2_{basename}_ult*.csv'))[0]
    data = pd.read_csv(csv_file, index_col=0)
    ult_features = clean_data(data).astype(float)

    # apply smoothing filters to dlc
    if butter:
        ult_features = apply_butter(ult_features)
    if med:
        ult_features = apply_medfilt(ult_features)

    return ult_features

def apply_sync(ult, ult_fps, wav, wav_sr, dlc):
    """all should be aligned at begining, need to trim the ends and return"""

    ult_dur = ult.shape[0] / ult_fps
    wav_dur = wav.shape[0] / wav_sr
    dlc_dur = dlc.shape[0] / ult_fps

    min_sec = np.min((ult_dur, wav_dur, dlc_dur))
    ult_end = int(round(ult_fps * min_sec))
    wav_end = int(round(wav_sr * min_sec))

    return ult[:ult_end], wav[:wav_end], dlc[:ult_end]


In [192]:
class TestFile:
    def __init__(self, basename, ult, wav, sr, dlc_ult):
        self.basename = basename
        self.ult = ult
        self.wav = wav
        self.sr = sr
        self.dlc = dlc_ult
        self.aud_scalar = None
        self.ult_scalar = None
        self.dlc_scalar = None
        self.aud_feat = None
        self.window = None
        self.lstm = None
        self.get_butter()
        self.get_med()
        self.get_bothfilt()

    def get_butter(self):
        """apply butterworth filter to each column (body part)"""
        filtered = np.empty_like(self.dlc)
        fc = 10  # Cut-off frequency of the filter
        w = fc / (60 / 2)  # Normalize the frequency
        b, a = signal.butter(5, w, 'low')

        for col in range(self.dlc.shape[1]):
            filtered[:, col] = signal.filtfilt(b, a, self.dlc[:, col])

        self.butter = filtered

    def get_med(self):
        """apply median filter to each column (body part)"""
        filtered = np.empty_like(self.dlc)
        for col in range(self.dlc.shape[1]):
            filtered[:, col] = signal.medfilt(self.dlc[:, col], 7)

        self.medfilt = filtered

    def get_bothfilt(self):
        """apply median filter to each column (body part)"""
        filtered = np.empty_like(self.dlc)
        for col in range(self.dlc.shape[1]):
            filtered[:, col] = signal.medfilt(self.butter[:, col], 7)

        self.bothfilt = filtered


In [193]:
# choose path to file dir and choose files to open
path = 'speech2ult/predictions/test_data'
files = ['088_aud', '133_aud', '226_aud']
# change the file to create testfile for and run from here
# for file in files:
file = files[1]

In [194]:
# load in the data - ult, wav and dlc_ult
true_ult, offset = load_ult(os.path.join(path, file))
wav, sr = librosa.load(os.path.join(path, file + '.wav'), offset=offset, sr=20000)
true_dlc = load_dlc_ult(file, path)

print(true_ult.shape, true_dlc.shape)

(482, 64, 128) (345, 22)


In [195]:
# sync
true_ult, wav, true_dlc = apply_sync(true_ult, 60, wav, sr, true_dlc)
print(true_ult.shape, true_dlc.shape)

(345, 64, 128) (345, 22)


In [196]:
# 088_aud VAD 1.44 - 4.05 -> in frames 86:243
# 133_aud VAD 1.53 - 5.04 -> 91:303
# 226_aud VAD 1.44 - 2.73 -> 86:164
# want to keep pad with 12 frames 0.2s
# get time segments
# time_segments = detect_voice_activity(wav,sr)
# print(time_segments)

In [197]:
# apply VAD by hand with padding of 0.2s which is 12 frames
if file == '088_aud':
    s, e = 74, 255
    s1, e1 = int((1.44-0.2) * sr), int((4.05+0.2) * sr)
elif file == '133_aud':
    s, e = 79, 314
    s1, e1 = int((1.53-0.2) * sr), int((5.04+0.2) * sr)
elif file == '226_aud':
    s, e = 74, 176
    s1, e1 = int((1.44-0.2) * sr), int((2.73+0.2) * sr)

true_ult, wav, true_dlc = true_ult[s:e], wav[s1:e1], true_dlc[s:e]

In [198]:
# check duration
print(wav.shape[0] / sr, true_ult.shape[0] / 60)

3.91 3.9166666666666665


In [199]:
testfile = TestFile(file, true_ult, wav, sr, true_dlc)

In [200]:
# get audio features
def get_aud_feats(wav, deltas=True):
    """get the audio features from a loaded wav, return as (frames, features)"""
    feats = librosa.feature.mfcc(
        y=wav,
        sr=sr,
        n_mfcc=20,
        hop_length= int(sr*(1/60)),
        n_fft=int(sr*0.02)
    )
    if deltas:
        deltas = librosa.feature.delta(feats)
        feats = np.concatenate((feats,deltas))
    return feats.T

In [201]:
aud_feat = get_aud_feats(wav)
print(aud_feat.shape, true_ult.shape)

(235, 40) (235, 64, 128)


In [202]:
# we actually only need to normalize aud features - load in scalars from dict
dlc_dict = 'speech2ult/predictions/test_data/PARAM_DICTS_ffn_dlc_2022-08-03.pickle'
window_dict='speech2ult/predictions/test_data/PARAM_DICTS_ffn_w_2022-08-03.pickle'
with open(dlc_dict, 'rb') as file:
    dlc_params, aud_params = pickle.load(file)
with open(window_dict, 'rb') as file:
    ult_params, _ = pickle.load(file)

# get scalars and check size
aud_feat = aud_params['scalar'].transform(aud_feat)


In [203]:
testfile.aud_feat = aud_feat
testfile.aud_scalar = aud_params['scalar']
testfile.ult_scalar = ult_params['scalar']
testfile.dlc_scalar = dlc_params['scalar']


In [204]:
# format for window - keep number of frames same to true ult
def apply_window(aud_in):
    # get dims of aud_in
    frames, n_feats = aud_in.shape

    # create new aud array with space for window
    aud_out = np.empty((frames, 5, n_feats))

    # populate new arrays
    for i in range(2, frames-3):  # i the first frame in the window
        aud_out[i] = aud_in[i-2:i+3]

    # need to populate the edges 0, 1, -1, -2
    first = aud_in[0]
    last = aud_in[-1]
    aud_out[0,0], aud_out[0,1], aud_out[0, 2:] = first, first, aud_in[0:3]
    aud_out[1,0], aud_out[1, 1:] = first, aud_in[:4]
    aud_out[-1,:3], aud_out[-1, 3], aud_out[-1, 4]  = aud_in[-3:], last, last
    aud_out[-2,:4], aud_out[-2, 4]  = aud_in[-4:], last

    return aud_out.reshape((frames, 5 * n_feats))
testfile.window = apply_window(testfile.aud_feat)

In [205]:
# format for lstm - keep number of frames same to true ult
def apply_lb(aud_in, lb):
    # get dims of aud_in
    frames, n_feats = aud_in.shape
    if n_feats == 40:
        n_feats = 20
        aud_in = aud_in[:, :20]

    # create new aud array with space for lb
    aud_out = np.empty((frames, lb, n_feats))

    # populate new arrays
    for i in range(lb-1, frames):
        aud_out[i] = aud_in[i-lb+1:i+1]

    # need to populate the edges 0-8
    for i in range(lb-1):
        aud_out[i, :lb-1-i] = np.tile(aud_in[0], (lb-1-i, 1)) # padding
        aud_out[i, lb-1-i:] = aud_in[0:i+1]

    return aud_out

testfile.lstm = apply_lb(testfile.aud_feat, 10)


In [206]:
# pickle testfile
pick_path = os.path.join('/Users/jacobrosen/Desktop/Edinburgh/Dissertation/project_code/predictions/test_data', f'{testfile.basename}_testfile.pickle')
with open(pick_path, 'wb') as file:
    pickle.dump(testfile, file, pickle.HIGHEST_PROTOCOL)